# Prédiction de l’achalandage des stations Bixi

Ce projet vise à prédire le taux d’entrées et de sorties par tranche horaire dans les stations Bixi à l’aide de plusieurs modèles :

- **STGNN (Spatio-Temporal Graph Neural Network)** : réseau neuronal à graphes spatio-temporels, stations comme nœuds, arêtes basées sur la distance ou le temps de déplacement.
- **LSTM (Long Short-Term Memory)** : réseau neuronal récurrent adapté aux séries temporelles.
- **Modèle statistique** : régression ou analyse classique pour servir de baseline.

Nous comparerons la performance de ces modèles sur les données d’achalandage.

---

## 1. Importation des librairies et des données

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
# Pour apprentissage
#import torch
#import torch.nn as nn
#from torch.utils.data import DataLoader, Dataset
# Pour LSTM
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import LSTM, Dense
# Pour le modèle statistique
import statsmodels.api as sm
# Pour le graphe
# import networkx as nx

## 2. Exploration et préparation des données

- Nettoyage et visualisation

- Construction du graphe des stations

In [3]:
df = pd.read_csv("data/merged2018_reduced_dataset.csv", parse_dates=['start_date', 'end_date'])
df['start_date'] = pd.to_datetime(df['start_date'])
# print(df.get(0))
df_grouped = df.groupby([ pd.Grouper(key="start_date", freq="15min"),
                 pd.Grouper('start_station_code')
                 ]).count()

print(df_grouped.sort_values(by=['end_date'], ascending=False))

                                        end_date  end_station_code  \
start_date          start_station_code                               
2018-08-23 21:00:00 6184                      14                14   
2018-09-04 17:00:00 6184                      12                12   
2018-08-03 11:00:00 6131                      12                12   
2018-07-19 23:15:00 6155                      12                12   
2018-06-27 17:30:00 6184                      11                11   
...                                          ...               ...   
2018-11-15 23:45:00 6155                       1                 1   
                    6158                       1                 1   
                    6164                       1                 1   
                    6169                       1                 1   
2018-04-10 12:30:00 6237                       1                 1   

                                        duration_sec  
start_date          start_station_

## 3. Modélisation

### 3.1 Modèle statistique (baseline) 

In [2]:
df = pd.read_csv("data/merged2018_reduced_dataset.csv") 
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])
df['slot_15'] = df['end_date'].dt.floor('15T') # Tronquer à l'intervalle de 15 minutes le plus proche

# Calcul des arrivées par station et par intervalle de 15 minutes
arrivals = (
    df
    .groupby(['end_station_code', 'slot_15'])
    .size()
    .reset_index(name='arrivals')
)

# Calcul de la moyenne historique des arrivées par station, jour de la semaine, heure et minute
arrivals['dow'] = arrivals['slot_15'].dt.dayofweek   # 0=lundi
arrivals['hour'] = arrivals['slot_15'].dt.hour     # 0-23
arrivals['minute'] = arrivals['slot_15'].dt.minute   # 0,15,30,45

# Calcul de la moyenne historique des arrivées
historical_mean = (
    arrivals
    .groupby(['end_station_code', 'dow', 'hour', 'minute'])['arrivals']
    .mean()
    .reset_index()
    .rename(columns={'arrivals': 'mean_arrivals'})
)
# Fusion des données réelles avec les moyennes historiques
pred = arrivals.merge(
    historical_mean,
    on=['end_station_code', 'dow', 'hour', 'minute'],
    how='left'
)

from sklearn.metrics import mean_absolute_error, mean_squared_error
# Calcul des métriques MAE et RMSE
mae = mean_absolute_error(pred['arrivals'], pred['mean_arrivals'])
rmse = np.sqrt(mean_squared_error(pred['arrivals'], pred['mean_arrivals']))
print("MAE:", mae)
print("RMSE:", rmse)

MAE: 0.4473305597783349
RMSE: 0.6870355561229784


In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Chargement et préparation
df = pd.read_csv("data/merged2018_reduced_dataset.csv") 
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])
df['slot_15'] = df['end_date'].dt.floor('15T')

# SPLIT TEMPOREL SANS LEAKAGE
# Trier par date pour un split temporel propre
df = df.sort_values('end_date')

# Split temporel (80% train, 20% test)
split_date = df['end_date'].quantile(0.8)
train_df = df[df['end_date'] < split_date]
test_df = df[df['end_date'] >= split_date]

# Calcul des moyennes historiques SUR LE TRAIN SEULEMENT
train_arrivals = (
    train_df
    .groupby(['end_station_code', 'slot_15'])
    .size()
    .reset_index(name='arrivals')
)

train_arrivals['dow'] = train_arrivals['slot_15'].dt.dayofweek
train_arrivals['hour'] = train_arrivals['slot_15'].dt.hour
train_arrivals['minute'] = train_arrivals['slot_15'].dt.minute

historical_mean = (
    train_arrivals
    .groupby(['end_station_code', 'dow', 'hour', 'minute'])['arrivals']
    .mean()
    .reset_index()
    .rename(columns={'arrivals': 'mean_arrivals'})
)

# Prédictions sur le TEST
test_arrivals = (
    test_df
    .groupby(['end_station_code', 'slot_15'])
    .size()
    .reset_index(name='true_arrivals')
)

test_arrivals['dow'] = test_arrivals['slot_15'].dt.dayofweek
test_arrivals['hour'] = test_arrivals['slot_15'].dt.hour
test_arrivals['minute'] = test_arrivals['slot_15'].dt.minute

# Fusion avec moyennes historiques
pred = test_arrivals.merge(
    historical_mean,
    on=['end_station_code', 'dow', 'hour', 'minute'],
    how='left'
)

# Gestion des valeurs manquantes (pas dans le train)
# Moyenne globale de la station
station_global_mean = train_arrivals.groupby('end_station_code')['arrivals'].mean()

def fill_missing(row):
    if pd.isna(row['mean_arrivals']):
        return station_global_mean.get(row['end_station_code'], 0)
    return row['mean_arrivals']

pred['predicted_arrivals'] = pred.apply(fill_missing, axis=1)

# Évaluation
mae = mean_absolute_error(pred['true_arrivals'], pred['predicted_arrivals'])
rmse = np.sqrt(mean_squared_error(pred['true_arrivals'], pred['predicted_arrivals']))

print(f"MAE:  {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"Nombre de prédictions: {len(pred)}")


MAE:  0.4814
RMSE: 0.6990
Nombre de prédictions: 65662


### 3.2 Modèle statistique SARIMA et ARIMA

In [4]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_absolute_error, mean_squared_error

df = pd.read_csv("data/merged2018_reduced_dataset.csv")

df['end_date'] = pd.to_datetime(df['end_date'])
df['slot_15'] = df['end_date'].dt.floor('15T')

arrivals = (
    df.groupby(['end_station_code', 'slot_15'])
      .size()
      .reset_index(name='arrivals')
)


stations_to_test = [6017]     


def train_arima(ts):
    train_size = int(len(ts) * 0.8)
    train, test = ts[:train_size], ts[train_size:]

    model = ARIMA(train, order=(1,0,1)).fit()
    preds = model.forecast(len(test))

    mae = mean_absolute_error(test, preds)
    rmse = np.sqrt(mean_squared_error(test, preds))
    return mae, rmse


def train_sarima(ts, season=96):
    """
    96 = 24h * (4 slots per hour)
    """
    train_size = int(len(ts) * 0.8)
    train, test = ts[:train_size], ts[train_size:]

    model = SARIMAX(
        train,
        order=(1,0,1),
        seasonal_order=(1,0,1,season),
        enforce_stationarity=False,
        enforce_invertibility=False
    ).fit(disp=False)

    preds = model.forecast(len(test))

    mae = mean_absolute_error(test, preds)
    rmse = np.sqrt(mean_squared_error(test, preds))
    return mae, rmse


results = []

for st in stations_to_test:
    print(f"\n Station {st} ")

    station_df = arrivals[arrivals['end_station_code'] == st].copy()
    station_df = station_df.set_index("slot_15").sort_index()

    ts = station_df['arrivals']

    if len(ts) < 200:
        print("Pas assez de données → skip")
        continue

    mae_a, rmse_a = train_arima(ts)
    mae_s, rmse_s = train_sarima(ts)

    print(f"ARIMA  -> MAE={mae_a:.3f}, RMSE={rmse_a:.3f}")
    print(f"SARIMA -> MAE={mae_s:.3f}, RMSE={rmse_s:.3f}")

    results.append({
        "station": st,
        "ARIMA_MAE": mae_a,
        "ARIMA_RMSE": rmse_a,
        "SARIMA_MAE": mae_s,
        "SARIMA_RMSE": rmse_s,
    })


results_df = pd.DataFrame(results)
print("\n Résultats")
print(results_df)



 Station 6017 


c:\Users\Bineta\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\Bineta\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\Bineta\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:581: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  warnings.warn('A date index has been provided, but it has no'
c:\Users\Bineta\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:376: ValueWarning: No supported index is available. Prediction r

ARIMA  -> MAE=0.283, RMSE=0.424
SARIMA -> MAE=0.327, RMSE=0.452

 Résultats
   station  ARIMA_MAE  ARIMA_RMSE  SARIMA_MAE  SARIMA_RMSE
0     6017   0.283268    0.423796    0.326751     0.451709


### 3.2 Modèle LSTM

In [ ]:
# Modèle LSTM pour la prédiction des arrivées par station

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import torch
import tensorflow as tf

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



# Exemple : prédire pour une station donnée (ex : 6169)
station_id = 6169
df_station = arrivals[arrivals['end_station_code'] == station_id].sort_values('slot_15')

# Série temporelle des arrivées
series = df_station['arrivals'].values.reshape(-1, 1)

# Normalisation
scaler = MinMaxScaler()
series_scaled = scaler.fit_transform(series)

# Création des séquences pour LSTM
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 96  # 24h si 15min pas step
X, y = create_sequences(series_scaled, seq_length)

# Split train/test
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Modèle LSTM
model = Sequential([
    LSTM(32, input_shape=(seq_length, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse')

# Entraînement
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Prédiction et inverse transform
y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test)

# Évaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error
mae = mean_absolute_error(y_test_inv, y_pred_inv)
rmse = np.sqrt(mean_squared_error(y_test_inv, y_pred_inv))
print('MAE LSTM:', mae)
print('RMSE LSTM:', rmse)

Num GPUs Available: 0
TensorFlow version: 2.20.0
Num GPUs Available: 0
cpu
Epoch 1/20


c:\Users\olidr\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0096 - val_loss: 0.0053
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0096 - val_loss: 0.0053
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0093 - val_loss: 0.0052
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0093 - val_loss: 0.0052
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0093 - val_loss: 0.0052
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0093 - val_loss: 0.0052
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0093 - val_loss: 0.0053
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0093 - val_loss: 0.0053
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0093 - val_loss: 0.0053
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0093 - val_loss: 0.0053
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - loss: 0.0093 - val_loss: 0.0056
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step -

### 3.3 Modèle STGNN (Spatio-Temporal Graph Neural Network)

## 4. Comparaison des performances

- Métriques : RMSE, MAE, etc.
- Visualisation des résultats